# Checking enviroment states and action sets

In [1]:
import sys
import os.path
sys.path.append("..")
from PPO_Agent_merged import * #PPO_Agent_v2 PPO_Agent_with_Guided_AC
from ENV_DETAILS import *
from RUN_TESNORBOARD import *

# events_folder = "./logs_hyper"
# main("./logs_hyper")

2024-02-25 08:29:17.058625: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
if os.name == 'nt':
    main_hyper_dir = "D:\\Artificial_Intelligence\\Portfolio\\RL_updated\\MountainCar\\" # Windows
    conda_python_exec = 'C:\\Users\\yanie\\anaconda3\\envs\\ai_dev\\python.exe '# Windows
    logs_dir = main_hyper_dir+'Hyperparam_tuning\\' # Windows
else:
    main_hyper_dir = "/media/n/NewDisk/Artificial_Intelligence/Portfolio/RL_updated/MountainCar/" # Linux
    conda_python_exec = '/home/n/anaconda3/envs/ai_dev/bin/python '# Linux
    logs_dir = main_hyper_dir+'Hyperparam_tuning_a3c/' # Linux

ENV = "MountainCarContinuous-v0"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]
EPISODES = ENV_DETAILS[ENV]["EPISODES"]
SUCESS_CRITERIA_VALUE

90.0

In [3]:

env = gym.make(ENV)
env

<TimeLimit<OrderEnforcing<Continuous_MountainCarEnv<MountainCarContinuous-v0>>>>

In [4]:
env._max_episode_steps

999

In [5]:
s = env.observation_space.sample()
s

array([-0.6651659 , -0.03973157], dtype=float32)

In [6]:
env.observation_space, env.observation_space.shape[0]

(Box([-1.2  -0.07], [0.6  0.07], (2,), float32), 2)

In [7]:
s = env.reset()
s[0]

-0.40082148

In [8]:
env.action_space

Box(-1.0, 1.0, (1,), float32)

### Training/Hyperparam run

In [9]:
#%tensorboard --logdir ./logs_hyper

In [10]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "Training/fit_PPO_guided/"

In [14]:
if HYPERPARAM_TUNING:

    dir = r"Hyperparam_kt_ppo"
    project_name = "keras_tunning_ppo_guided"
  
    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/ppo_guided/", evaluation_epoch = env._max_episode_steps, training_steps = 700000,
                sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
                discount_min = 0.90, discount_max = 0.99,
                #discount = 0.99,
                policy_clip = 0.2,
                gae_min = 0.94, gae_max = 0.96,
                lr_agent_min = 0.00001, lr_agent_max = 0.001,
                entropy_factor_min = 0.001, entropy_factor_max = 0.1,
                dense_min = 32, dense_max = 256,
                environment_name=ENV,
                num_layers_agent = 2, training_epoch = 100,  normalize = False,
                memory_size= env._max_episode_steps
                ),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 40,
            # distribution_strategy= strategy,
            directory=dir,
            beta = 10,
            project_name=project_name
        )
    tuner.search(x=[0], y=[1])
else : 
    
        print("Acquiring parameters ....")

        training_steps = 1000000
        entropy_factor = 0.05
        discount = 0.99
        dense_units_agent = [128,128] #64, 32]
        num_layer_agent = 2

        model = run_training(training_steps,  discount, dense_units_agent,  num_layer_agent, writer, 
                      environment_name = ENV,return_agent = True, agent_lr= 0.0001, normalize = True,
                      gae_lambda = 0.95, entropy_coeff = entropy_factor, policy_clip = 0.2, training_epoch = 5)


Trial 40 Complete [01h 30m 03s]
total_train_reward: -80.14733135559602

Best total_train_reward So Far: -52.069832466313024
Total elapsed time: 4d 10h 04m 48s


In [12]:
exploration_tech = "soft"
hyperparam_combination=[]

for trials in tuner.oracle.get_best_trials(num_trials=50):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

Trial id :08 | Score :-52.069832466313024 --> {'discount': 0.9, 'gae_lambda': 0.95, 'agent_lr': 0.00010577186590904286, 'entropy_coeff': 0.043667988955104375, 'num_layer_agent': 1, 'dense_units_agent_0': 158, 'dense_units_agent_1': 175}
Trial id :05 | Score :-62.91236911926401 --> {'discount': 0.97, 'gae_lambda': 0.96, 'agent_lr': 0.00036367149574077746, 'entropy_coeff': 0.0695328842691358, 'num_layer_agent': 1, 'dense_units_agent_0': 86, 'dense_units_agent_1': 125}
Trial id :04 | Score :-65.54745874504796 --> {'discount': 0.9500000000000001, 'gae_lambda': 0.94, 'agent_lr': 0.00016471050757726357, 'entropy_coeff': 0.02718591705211787, 'num_layer_agent': 2, 'dense_units_agent_0': 109, 'dense_units_agent_1': 51}
Trial id :18 | Score :-67.64265664538723 --> {'discount': 0.91, 'gae_lambda': 0.94, 'agent_lr': 4.697922200117869e-05, 'entropy_coeff': 0.06415108752076276, 'num_layer_agent': 2, 'dense_units_agent_0': 73, 'dense_units_agent_1': 226}
Trial id :16 | Score :-67.84943951772239 --> {

In [15]:
val_env = gym.make(ENV)#, render_mode = "rgb_array"
dir = r"Hyperparam_kt_ppo"
for trials in tuner.oracle.get_best_trials(num_trials=1):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)    
    training_steps = 1000000
    entropy_factor = trials.hyperparameters.values["entropy_coeff"]
    discount = trials.hyperparameters.values["discount"]
    gae = trials.hyperparameters.values["gae_lambda"]
    try:
        policy_clip = trials.hyperparameters.values["policy_clip"]
    except :
        policy_clip = 0.2
    
    lr_actor=  trials.hyperparameters.values["lr_actor"]
    lr_critic=  trials.hyperparameters.values["lr_critic"]
    lr_model=  trials.hyperparameters.values["lr_model"]
    
    try:
        n_dense_layers_actor = trials.hyperparameters.values["n_dense_layers_actor"]
    except : 
        n_dense_layers_actor = 1
        
    try:
        n_dense_layers_critic = trials.hyperparameters.values["n_dense_layers_critic"]
    except:
        n_dense_layers_critic = 1
        

    dense_layers_actor = []
    for i in range(n_dense_layers_actor):
        dense_layers_actor.append(trials.hyperparameters.values['dense_units_act_'+str(i)])

    dense_layers_critic = []
    for i in range(n_dense_layers_critic):
        dense_layers_critic.append(trials.hyperparameters.values['dense_units_crit_'+str(i)])

    
    n_dense_layers_model = 1
    dense_layers_model = []
    for i in range(n_dense_layers_model):
        dense_layers_model.append(trials.hyperparameters.values['n_dense_layers_model'+str(i)])

    model = run_training(
        training_steps = training_steps,   
            discount = discount,
            dense_units_act = dense_layers_actor, 
            dense_units_crit = dense_layers_critic,
            dense_units_model = dense_layers_model,
            num_layer_a = n_dense_layers_actor,
            num_layer_c = n_dense_layers_critic,
            num_layer_m = n_dense_layers_model,
            writer = writer,  
            save_factor=50000, 
            sucess_criteria_epochs =SUCESS_CRITERIA_EPOCH, 
            sucess_criteria_value = SUCESS_CRITERIA_VALUE,
            environment_name=ENV,
            reward_scaler = 1, 
            evaluation_epoch = env._max_episode_steps,
            return_agent = True,
            lr_actor= lr_actor, 
            lr_critic= lr_critic,
            lr_model= lr_model,
            gae_lambda=gae,
            training_epoch= 200,
            entropy_coeff= entropy_factor,
            policy_clip = policy_clip,
            memory_size= env._max_episode_steps,
            id = int(trials.trial_id))
        
final_rewards = final_evaluation(model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./ppo_guided_"+exploration_tech+"_video.mp4", env= ENV)

Trial id :08 | Score :-52.069832466313024 --> {'discount': 0.9, 'gae_lambda': 0.95, 'agent_lr': 0.00010577186590904286, 'entropy_coeff': 0.043667988955104375, 'num_layer_agent': 1, 'dense_units_agent_0': 158, 'dense_units_agent_1': 175}


KeyError: 'lr_actor'